In [ ]:
# Download nro-delegated into a pandas DataFrame
import pandas as pd
import requests
from io import StringIO
link = "https://ftp.ripe.net/pub/stats/ripencc/nro-stats/latest/nro-delegated-stats"


response = requests.get(link)
data = StringIO(response.text)
df = pd.read_csv(data, delimiter='|', comment='#', header=None,
                names=['registry', 'cc', 'type', 'start', 'value', 'date', 'status', 'extensions', 'source'])

df = df[df['type'] == 'asn']
df = df[df['status'] == 'assigned']

# Expand the Dataframe with each ASN in the range
def expand_asn_range(row):
    start_asn = int(row['start'])
    count = int(row['value'])
    return pd.DataFrame({
        'asn': range(start_asn, start_asn + count),
        'cc': row['cc'],
        'date': row['date'],
    })

expanded_df = pd.concat([expand_asn_range(row) for _, row in df.iterrows()], ignore_index=True)


/tmp/ipykernel_417207/4158016487.py:10: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data, delimiter='|', comment='#', header=None,


In [11]:
import tqdm as tqdm
tqdm.tqdm.pandas()

bgp_view_link = "https://api.bgpview.io/asn/{}"
bgp_view_link_prefixes = "https://api.bgpview.io/asn/{}/prefixes"

import requests

def apply_bgpview_api(asn):
    out = {"asn": asn}
    try:
        resp = requests.get(bgp_view_link.format(asn))
        if resp.status_code != 200:
            raise Exception(f"Error fetching ASN {asn}: {resp.status_code}")
        d1 = resp.json()
    except Exception as e:
        print(e)
        d1 = None
    try:
        d2 = requests.get(bgp_view_link_prefixes.format(asn)).json()
    except Exception as e:
        print(e)    
        d2 = None
    if d1 and "data" in d1:
        out.update(d1["data"])
    if d2 and "data" in d2:
        out.update(d2["data"])
    return out

expanded_df = expanded_df.progress_apply(
    lambda row: {**row, **apply_bgpview_api(row['asn'])}, axis=1, result_type='expand'
)

  0%|          | 2/119688 [00:00<6:59:35,  4.75it/s]

Expecting value: line 1 column 1 (char 0)
Error fetching ASN 2: 429


  0%|          | 4/119688 [00:01<10:10:44,  3.27it/s]

Error fetching ASN 3: 429
Expecting value: line 1 column 1 (char 0)


  0%|          | 5/119688 [00:01<11:19:20,  2.94it/s]

...

  0%|          | 12/119688 [00:02<5:36:22,  5.93it/s]

Expecting value: line 1 column 1 (char 0)
Error fetching ASN 11: 429
Expecting value: line 1 column 1 (char 0)


  0%|          | 12/119688 [00:02<8:05:23,  4.11it/s]


Error fetching ASN 12: 429


KeyboardInterrupt: 

In [ ]:
expanded_df['asn'].dtype